In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from keras.layers import Input, Dense, Flatten,Activation
from keras.layers import Conv1D, MaxPooling1D,Dropout,LSTM
from keras.layers import Embedding

from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding

In [4]:
df = pd.read_csv('../data/df_for_train_classifire.csv', index_col=0)

In [5]:
df.head(2)

,type,text,label
0,2,амаравкэваратэн,таа’койӈын
1,2,йъйыӄык,ныӄэԓпэратӄэн


In [7]:
df.type.value_counts()[:5]

2    26473
3    20768
4    15301
5    10986
6     7741
Name: type, dtype: int64

In [85]:
result_dict = dict()

all_types = pd.unique(df.type).tolist()
for id_t, t in enumerate(all_types):
    print('Current type', id_t, 'All type', len(all_types))
    tmp_df = df[df.type == t]
    tmp_df = tmp_df.drop(['type'], axis=1)

    result_dict[t] = dict()
    result_dict[t]['le'] = LabelEncoder()
    tmp_df['label'] = result_dict[t]['le'].fit_transform(tmp_df['label'])

    labels_len = len(pd.unique(tmp_df['label']).tolist())

    result_dict[t]['tokenizer'] = Tokenizer()
    result_dict[t]['tokenizer'].fit_on_texts(tmp_df.text.tolist())
    result_dict[t]['vocab_size'] = len(result_dict[t]['tokenizer'].word_index) + 1

    encoded_docs = result_dict[t]['tokenizer'].texts_to_sequences(tmp_df.text.tolist())
    
    padded_docs = pad_sequences(encoded_docs, maxlen=t-1, padding='post')

    labels = to_categorical(np.asarray(tmp_df['label'].tolist()))

    

    # result_dict[t]['model'] = Sequential()
    # result_dict[t]['model'].add(Embedding(result_dict[t]['vocab_size'], 10, input_length=t-1))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dropout(0.1))
    # result_dict[t]['model'].add(Flatten())
    # result_dict[t]['model'].add(Dense(30))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dense(labels_len))
    # result_dict[t]['model'].add(Activation('softmax'))
    # result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # result_dict[t]['model'].fit(padded_docs, labels, epochs=5, verbose=1)

    result_dict[t]['model'] = Sequential()
    result_dict[t]['model'].add(Embedding(result_dict[t]['vocab_size'], 20, input_length=t-1))
    result_dict[t]['model'].add(LSTM(64))
    result_dict[t]['model'].add(Dense(labels_len, activation='softmax'))
    result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    result_dict[t]['model'].fit(padded_docs, labels, epochs=10, verbose=1)

======] - 0s 689us/step - loss: 4.8248 - acc: 0.0580
Epoch 3/10
138/138 [==============================] - 0s 734us/step - loss: 4.8211 - acc: 0.0725
Epoch 4/10
138/138 [==============================] - 0s 847us/step - loss: 4.8174 - acc: 0.0797
Epoch 5/10
138/138 [==============================] - 0s 662us/step - loss: 4.8128 - acc: 0.0870
Epoch 6/10
138/138 [==============================] - 0s 696us/step - loss: 4.8070 - acc: 0.0870
Epoch 7/10
138/138 [==============================] - 0s 678us/step - loss: 4.7988 - acc: 0.1159
Epoch 8/10
138/138 [==============================] - 0s 672us/step - loss: 4.7863 - acc: 0.1232
Epoch 9/10
138/138 [==============================] - 0s 701us/step - loss: 4.7621 - acc: 0.1304
Epoch 10/10
138/138 [==============================] - 0s 677us/step - loss: 4.7020 - acc: 0.0942
Current type 17 All type 37
Epoch 1/10
99/99 [==============================] - 11s 110ms/step - loss: 4.5012 - acc: 0.0101
Epoch 2/10
99/99 [============================

In [86]:
f = open('output.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/dev.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = 'йъйыӄык' # TODO calc most freq word
        else:
            exits_text = ' '.join(tst_tokens[1:i+1])
            predicted_second = result_dict[i+1]['model'].predict(pad_sequences(result_dict[i+1]['tokenizer'].texts_to_sequences([exits_text]), maxlen=i, padding='post'))
            predicted_second = np.argmax(predicted_second)
            predicted_second = result_dict[i+1]['le'].inverse_transform([predicted_second])
            predicted_second = predicted_second[0]
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction
            output.append(predicted_second)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

100%|██████████| 1000/1000 [01:31<00:00, 10.87it/s]


In [87]:
hits

31